# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name:

Student ID: 

Email:

In [2]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [6]:
data = pd.read_csv("2008_csv.bz2")

In [61]:
airports = pd.read_csv("airports.csv")

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
data.describe(include='all')

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7009728,7.009728e+06,6926363,6.855029e+06,7.008884e+06,6.855029e+06,6.855029e+06,6.873482e+06,7009728,7009728,7.009728e+06,6.858079e+06,6.872670e+06,7.009728e+06,137434,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,5373,NaN,NaN,NaN,NaN,NaN,303,304,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN,NaN,N476HA,NaN,NaN,NaN,NaN,NaN,ATL,ATL,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1201754,NaN,4701,NaN,NaN,NaN,NaN,NaN,414513,414521,NaN,NaN,NaN,NaN,54904,NaN,NaN,NaN,NaN,NaN,NaN
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,NaN,2.224200e+03,NaN,1.273224e+02,1.288668e+02,1.040186e+02,8.168452e+00,9.972570e+00,NaN,NaN,7.263870e+02,6.860852e+00,1.645305e+01,1.960618e-02,NaN,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,NaN,1.961716e+03,NaN,7.018731e+01,6.940974e+01,6.743980e+01,3.850194e+01,3.531127e+01,NaN,NaN,5.621018e+02,4.933649e+00,1.133280e+01,1.386426e-01,NaN,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,NaN,1.000000e+00,NaN,1.200000e+01,-1.410000e+02,0.000000e+00,-5.190000e+02,-5.340000e+02,NaN,NaN,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,NaN,6.220000e+02,NaN,7.700000e+01,8.000000e+01,5.500000e+01,-1.000000e+01,-4.000000e+00,NaN,NaN,3.250000e+02,4.000000e+00,1.000000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,NaN,1.571000e+03,NaN,1.100000e+02,1.100000e+02,8.600000e+01,-2.000000e+00,-1.000000e+00,NaN,NaN,5.810000e+02,6.000000e+00,1.400000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,NaN,3.518000e+03,NaN,1.570000e+02,1.590000e+02,1.320000e+02,1.200000e+01,8.000000e+00,NaN,NaN,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00,NaN,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01


In [88]:
#1
print('Most frequent cancel:',data['CancellationCode'].value_counts().idxmax())
print('----------')
print('Minimal distance:', data['Distance'].min(), 'miles')
print('Maximal distance:', data['Distance'].max(), 'miles')
print('Average distance:', int(data['Distance'].mean()), 'miles')
print('----------')


#2
minFlights = data.loc[data['Distance'] == data['Distance'].min()]
print(minFlights[['DayofMonth', 'Month', 'Year','FlightNum']])
print('----------')


#3
minFlightsNum = []
minFlightsNum = minFlights['FlightNum'].get_values()

for i in minFlightsNum:
    flights = data.loc[data['FlightNum'] == i]
    print(flights[['FlightNum', 'DayofMonth', 'Month', 'Year', 'Distance']][:10], '\n')
    print('Different flights count:')
    print(flights['Distance'].value_counts(), '\n\n')
print('----------')


#4
airport = data['Origin'].value_counts().idxmax()
print('Most frequent Airport:\n')
print(airports[['iata', "airport","city","state","country"]].loc[airports['iata'] == airport])
print('----------')


#5
airportFlights = data.groupby('Origin').agg('mean')
airtime = airportFlights.loc[airportFlights['AirTime'] == airportFlights["AirTime"].max()]
airtime = airtime.reset_index()

print('Airport with the highest average airtime:\n')
print(airtime[['Origin','AirTime']], '\n')
airport = airtime['Origin'].get_values()[0]
print(airports[['iata', "airport","city","state","country"]].loc[airports['iata'] == airport])
print('----------')


#6
airportDelays = data.groupby('Origin').filter(lambda x: len(x) >= 1000)
airportDelays =  airportDelays.groupby('Origin')
delays = airportDelays.DepDelay.agg(lambda x: (x > 0).sum() / len(x))
delays = delays.loc[delays == delays.max()]
print('Airport with the highest delays:\n')
print(delays, '\n')
airport = delays.keys()[0]
print(airports[['iata', "airport","city","state","country"]].loc[airports['iata'] == airport])
print('----------')


Most frequent cancel: B
----------
Minimal distance: 11 miles
Maximal distance: 4962 miles
Average distance: 726 miles
----------
         DayofMonth  Month  Year  FlightNum
2547298          15      5  2008       4988
4392215          10      8  2008       5572
----------
        FlightNum  DayofMonth  Month  Year  Distance
151771       4988           3      1  2008       834
152430       4988           4      1  2008       834
153088       4988           5      1  2008       834
153545       4988           6      1  2008       834
154096       4988           7      1  2008       834
154750       4988           8      1  2008       834
155308       4988           9      1  2008       834
155962       4988          10      1  2008       834
156627       4988          11      1  2008       834
157287       4988          12      1  2008       834 

Different flights count:
834    130
903     59
321     18
223     11
712      4
414      1
96       1
11       1
Name: Distance, dtype: int64 

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [4]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [5]:
data.shape

(32769, 10)

In [6]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [9]:
# число значений у признаков
for col_name in data.columns:
    print (col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.